In [1]:
import json
import os
import pickle
from pathlib import Path
from joblib import Parallel, delayed
from statistics import mode

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import rioxarray

from xrspatial import focal, slope
import seaborn as sns
from tqdm import tqdm
from joblib_progress import joblib_progress
from xrspatial.multispectral import ndvi, savi
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay)
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss


In [2]:
# paths
helena_path = Path.cwd().parent / 'data' / 'helena'
feature_dir = helena_path / 'features'

In [4]:
# get paths for features using 100 m geomorphons
parquets = [p for p in os.listdir(feature_dir) if p.endswith('_100.parquet')]
parquets.sort()

parquets

['features_2018_crowns_100.parquet']

We will use the model which was tuned and trained in `src/mortality_classification.ipynb`.  It was pickled.

In [5]:
# load model created in src/mortality_classification.ipynb
pickle_path = Path.cwd() / 'RF_model.sav'
model = pickle.load(open(pickle_path, 'rb'))

Now we will make model predictions for the samples and create a timeseries of survival probabilities for each sample over the years for which we have NAIP data.

In [7]:
predictions = []
for f in parquets:
    print(f'-------{f}----------')
    
    # get year and geomorphon radius
    split_fname = f.split('_')
    y = split_fname[1]
    r = split_fname[3].split('.')[0]
    
    # read parquet, make input feature df (X)
    df = pd.read_parquet(feature_dir / f)
    cols = list(model.feature_names_in_)
    X = df[cols]
    y = pd.DataFrame()
    y['UniqueID'] = df['UniqueID']
    y[f'pred_{y}'] = model.predict_proba(X)[:, 1]
    predictions.append(y)
    
predictions = [predictions[0].join(df_, on='UniqueID') for df_ in predictions[1:]][0]


-------features_2018_crowns_100.parquet----------


IndexError: list index out of range

In [36]:
df = pd.read_parquet(feature_dir / f)
cols = list(model.feature_names_in_)
X = df[cols]
X.isin([np.inf, -np.inf]).sum()

#X.replace([np.inf, -np.inf], np.nan).dropna()

n_mean        0
rgi_mean      0
r_mean        0
n50           0
r30           0
rgi40         0
r20           0
n30           0
rgi50         0
n60           0
b20           0
g_mean        0
b30           0
g40           0
savi_std     93
ndvi_mean    93
savi_mean    93
b_std         0
r10           0
b_mean        0
lum70         0
rgi_std       0
n_std         0
lum30         0
n20           0
lum50         0
g20           0
r_std         0
lum20         0
lum60         0
g30           0
lum40         0
ndvi_std     93
n40           0
b40           0
g_std         0
r40           0
b50           0
lum10         0
lum80         0
n70           0
b10           0
r70           0
rgi60         0
g100          0
n10           0
rgi90         0
g90           0
n100          0
lum90         0
r50           0
b60           0
g50           0
dtype: int64